In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/spark/pagecounts-20160101-000000_parsed.out.zip"

Archive:  /content/drive/MyDrive/spark/pagecounts-20160101-000000_parsed.out.zip
  inflating: pagecounts-20160101-000000_parsed.out  


In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

In [6]:
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [7]:
!pip install -q findspark

In [8]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [9]:
from pyspark import SparkConf 
from pyspark.context import SparkContext 

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

**problem** **one** (we have two version for it ⚛)

In [10]:
def parse(line):
  cols=line.split(' ')
  codeCol=cols[0]
  pagetitle=cols[1]
  pageHits=cols[2]
  pageSize=cols[3]
  return(codeCol,pagetitle,pageHits,pageSize)

rdd=sc.textFile("/content/pagecounts-20160101-000000_parsed.out",minPartitions=0).map(parse).take(10)
#rdd=sc.parallelize(rdd).saveAsTextFile('oneV1.Txt')
rdd

[('aa', '271_a.C', '1', '4675'),
 ('aa', 'Category:User_th', '1', '4770'),
 ('aa', 'Chiron_Elias_Krase', '1', '4694'),
 ('aa', 'Dassault_rafaele', '2', '9372'),
 ('aa', 'E.Desv', '1', '4662'),
 ('aa', 'File:Wiktionary-logo-en.png', '1', '10752'),
 ('aa', 'Indonesian_Wikipedia', '1', '4679'),
 ('aa', 'Main_Page', '5', '266946'),
 ('aa', 'Requests_for_new_languages/Wikipedia_Banyumasan', '1', '4733'),
 ('aa', 'Special:Contributions/203.144.160.245', '1', '5812')]

In [ ]:
if __name__ == "__main__":
 data = sc.textFile("/content/pagecounts-20160101-000000_parsed.out",minPartitions=0)
 sc.parallelize(data.take(10)).saveAsTextFile('one.txt')

**Problem two**

In [ ]:
def parseLine(line):
  cols = line.split(" ")
  pagesize=float(cols[3])
  v=0
  return (v,pagesize)

data = sc.textFile("/content/pagecounts-20160101-000000_parsed.out",minPartitions=0)
pagesize=data.map(parseLine)

maxvalue = pagesize.reduceByKey(lambda x , y: max(x,y))
minvalue=pagesize.reduceByKey(lambda x , y: min(x,y))

minvalue.collect()
#maxvalue.collect()

[(0, 0.0)]

In [ ]:
def parseLine(line):
 cols = line.split(" ")
 pagesize=float(cols[3])
 return (pagesize)
pagesize=data.map(parseLine).mean()
pagesize

132239.5695744666

**Problem three**

In [ ]:
def parseLine(line):
  cols = line.split(" ")
  codeCol=cols[0]
  pagetitle=cols[1]
  return (codeCol,pagetitle)
pagesize=data.map(parseLine)
result=pagesize.filter(lambda x : (x[1].startswith('The')) & ('en' != x[0]))
result.count()

10292

**problem 5**

In [ ]:
def parseLine(line):
 cols = line.split(" ")
 pagetitle=cols[1]
 return (pagetitle)
pagesize=data.map(parseLine)
values=pagesize.map(lambda x: (x,1)).groupByKey()
result=values.mapValues(sum).map(lambda x:(x[0],x[1])).sortByKey(False).take(1)
print(result)

[(118, 'water')]


**problem 4**

In [ ]:
import re
def parse1(line):
 cols = line.split(" ")
 pagetitle=cols[1]
 result = re.sub(r'[^a-zA-Z_]','', pagetitle).lower()
 return result

def parse2(line):
 cols = line.split("_")
 return cols
pagetitle=data.map(parse1)
formated=pagetitle.map(parse2)
result=formated.flatMap(lambda x:x).distinct().count()bahaa_123
print(result)

1354511
